In [170]:
import pandas as pd
import CaboCha
import os

In [73]:
df = pd.read_table('wrime/wrime-ver1.tsv', encoding='utf-8')
df.shape

(43200, 44)

In [8]:
df['score_sum'] = df['Avg. Readers_Joy'] + df['Avg. Readers_Sadness'] + df['Avg. Readers_Anticipation'] + df['Avg. Readers_Surprise'] + df['Avg. Readers_Anger'] + df['Avg. Readers_Fear'] + df['Avg. Readers_Disgust'] + df['Avg. Readers_Trust']

In [4]:
count = 0

In [56]:
print(df.loc[df['score_sum']==0].iat[count,0] )
count += 1


出かけたときの「あれ食べたい」「これ食べたい」には負けてしまう。


In [14]:
print(df.loc[df['Avg. Readers_Trust'] > 1].iat[count,0])
count +=1

誰か鍋食べにいこお～または鍋に呼んでください！味噌鍋とかチゲとか濃い～のん食べたい！あ、カレー鍋でも可… 笑


In [159]:
wrime_df = pd.read_table('wrime/wrime-ver1.tsv', encoding='utf-8')
#c = CaboCha.Parser()
c = CaboCha.Parser('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
class DicElem:
    def __init__(self, lemma, **kwargs):
        self.lemma = lemma
        self.score = {
            "Joy":0,
            "Sadness":0,
            "Anticipation":0,
            "Surprise":0,
            "Anger":0,
            "Fear":0,
            "Disgust":0,
            "Trust":0
        }
        self.frequency = 0
        self.add_score(**kwargs)
    def add_score(self, **kwargs):
        for key, item in kwargs.items():
            if(type(item) is int):
                self.score[key] += item
        self.frequency += 1
        
dic = dict()

In [160]:
for i, row in wrime_df[:].iterrows():
    tree = c.parse(row['Sentence'])
    score = {
        "Joy":row['Avg. Readers_Joy'],
        "Sadness":row['Avg. Readers_Sadness'],
        "Anticipation":row['Avg. Readers_Anticipation'],
        "Surprise":row['Avg. Readers_Surprise'],
        "Anger":row['Avg. Readers_Anger'],
        "Fear":row['Avg. Readers_Fear'],
        "Disgust":row['Avg. Readers_Disgust'],
        "Trust":row['Avg. Readers_Trust']
    }
    for s_index in range(tree.size()):
        token = tree.token(s_index)
        features = token.feature.split(',')
        lemma = features[6]
        wordcategory = features[0]
        subwordcategory = features[1]
        if(lemma in dic.keys()):
            dic[lemma].add_score(**score)
        else:
            dic[lemma] = DicElem(lemma, **score)

In [161]:
to_df_dic = {
    'lemma':[],
    "Joy":[],
    "Sadness":[],
    "Anticipation":[],
    "Surprise":[],
    "Anger":[],
    "Fear":[],
    "Disgust":[],
    "Trust":[],
    "Sum":[],
    "ave_Joy":[],
    "ave_Sadness":[],
    "ave_Anticipation":[],
    "ave_Surprise":[],
    "ave_Anger":[],
    "ave_Fear":[],
    "ave_Disgust":[],
    "ave_Trust":[],
    "ave_Sum":[],
    "norm":[],
    "frequency":[]
}
for key, item in dic.items():
    to_df_dic['lemma'].append(key)
    frequency = item.frequency
    to_df_dic['frequency'].append(frequency)
    sum = 0
    norm = 0
    for s_key, s_item in item.score.items():
        to_df_dic[s_key].append(s_item)
        sum += s_item
        norm += (s_item/frequency) ** 2
        to_df_dic[f'ave_{s_key}'].append(s_item/frequency)
    norm = norm ** 0.5
    to_df_dic['Sum'].append(sum)
    to_df_dic['ave_Sum'].append(sum/frequency)
    to_df_dic['norm'].append(norm)
    

In [162]:
word_df = pd.DataFrame(to_df_dic)
word_df.to_csv('wrime_word.csv', encoding='utf-8', index=0)

In [86]:
word_df.loc[word_df['frequency'] > 10].sort_values('norm', ascending=False).to_csv('emotional_words.csv', encoding='utf-8', index=False)

In [177]:
target_words = ['嬉しい','興奮', '幸せ', '驚き', 'イライラ', '怖い', '嫌', '憂鬱', '疲れ', '疲れる', '眠い', '息抜き', '休息' , 'だらける', '安心']
res = word_df.loc[word_df['lemma'].isin(target_words), ['lemma','frequency']]
target_words = list(res['lemma'])
print(res)
print([i for i in target_words if not i in list(res['lemma'])])

      lemma  frequency
46       眠い        295
209     嬉しい        337
217      幸せ        332
388      安心         68
1703      嫌        258
1746    疲れる        332
2001     興奮         25
2013   イライラ         84
2637     怖い        322
5877     疲れ         56
5888     驚き         21
8951    息抜き          9
20295    憂鬱         30
23381    休息          3
25195  だらける          8
[]


In [179]:
#word_df.loc[word_df['lemma']=='安心'].at[word_df.index[0],'lemma']
can_override = False
for target_word in target_words:
    if(not can_override and f'{target_word}.csv' in os.listdir('near_neologd')):
        print(f'skip {target_word}')
        continue
    else:
        print(f'write {target_word}')
    rows = [
        "ave_Joy",
        "ave_Sadness",
        "ave_Anticipation",
        "ave_Surprise",
        "ave_Anger",
        "ave_Fear",
        "ave_Disgust",
        "ave_Trust",
        "ave_Sum",
    ]
    tmp = word_df.loc[word_df['lemma']==target_word]
    try:
        target_vector = dict(tmp.loc[tmp.index[0], rows])
    except:
        print(target_word, 'no')
        continue
    #print(target_vector)

    def cos_similarity(r1, r2):
        norm1 = 0
        norm2 = 0
        inner = 0
        for i in rows:
            norm1 += r1[i]**2 
            norm2 += r2[i]**2
            inner += r1[i] * r2[i]
        if(norm1 == 0 or norm2 == 0):
            return 999
        else:
            return inner / (norm1**0.5 * norm2**0.5)

    cos_df = word_df.copy()
    for index, row in cos_df.iterrows():
        cos_df.at[index, f'cos_with_{target_word}'] = cos_similarity(target_vector, row)
    cos_df.loc[cos_df['frequency'] > 10].sort_values(f'cos_with_{target_word}', ascending=False).loc[:, ['lemma',f'cos_with_{target_word}', 'frequency']].to_csv(f'near_neologd/{target_word}.csv')


skip 眠い
skip 嬉しい
skip 幸せ
skip 安心
skip 嫌
skip 疲れる
skip 興奮
skip イライラ
skip 怖い
skip 疲れ
skip 驚き
skip 息抜き
skip 憂鬱
skip 休息
skip だらける


In [148]:
cos_df.loc[cos_df['frequency'] > 10].sort_values(f'cos_with_{target_word}', ascending=False).loc[:, ['lemma',f'cos_with_{target_word}', 'frequency']].to_csv(f'near/{target_word}.csv')

In [149]:
1.7320508/2

0.8660254